## READ THE DATA

In [1]:
def ExpressionStringToList(expressionString):
    return [char for char in expressionString if char != ' ']

def ExpressionListToString(expressionList):
    return ' '.join(expressionList)

In [2]:
fileName = 'input.txt'

with open(fileName) as openFile:
    expressionList = [ExpressionStringToList(line.rstrip()) for line in openFile]

print("Number of expressions to process:", len(expressionList))

Number of expressions to process: 370


## PART 1

In [3]:
## Find index of the right parenthesis that closes the selected left one
def RightParenthesisIndex(expr, leftParenthesisIndex):
    if expr[leftParenthesisIndex] != '(':
        print("ERROR: it seems you thought", expr[leftParenthesisIndex], 'was a left parenthesis in expression', ExpressionListToString(expr), ".")
        return -1
    
    parenthesisCount = 1
    currentIndex = leftParenthesisIndex
    
    while parenthesisCount != 0:
        currentIndex += 1
        if expr[currentIndex] == '(':
            parenthesisCount += 1
        elif expr[currentIndex] == ')':
            parenthesisCount -= 1
    
    return currentIndex

## Vice versa
def LeftParenthesisIndex(expr, rightParenthesisIndex):
    if expr[rightParenthesisIndex] != ')':
        print("ERROR: it seems you thought", expr[rightParenthesisIndex], 'was a right parenthesis in expression', ExpressionListToString(expr), ".")
        return -1
    
    parenthesisCount = 1
    currentIndex = rightParenthesisIndex
    
    while parenthesisCount != 0:
        currentIndex -= 1
        if expr[currentIndex] == ')':
            parenthesisCount += 1
        elif expr[currentIndex] == '(':
            parenthesisCount -= 1
    
    return currentIndex

In [4]:
def ProcessLongformExpression(expr):
    total = 0
    operator = '+'
    skipUntil = -1
    
    for index, element in enumerate(expr):
        if index < skipUntil:
            continue
        
        if element.isdigit():
            element = int(element)
            total = OperateOnTotal(total, operator, element)
        elif element in ['+', '*']:
            operator = element
        elif element == '(':
            indexOfMatchingRightParenthesis = RightParenthesisIndex(expr, index)
            totalOfParenthesis = ProcessLongformExpression(expr[index+1:indexOfMatchingRightParenthesis])
            total = OperateOnTotal(total, operator, totalOfParenthesis)
            skipUntil = indexOfMatchingRightParenthesis + 1
        else:
            print("ERROR: element", element, "in expression", ExpressionListToString(expr), "not recognized.")
            return -1
    
    return total

In [5]:
def OperateOnTotal(total, operator, value):
    value = int(value)
    if operator == '+':
        total += value
    elif operator == '*':
        total *= value
    else:
        print("ERROR: operator", operator, "not recognized.")
        total = -1
    
    return total

In [6]:
expressionTotal = 0

for expression in expressionList:
    expressionTotal += ProcessLongformExpression(expression)

print("Sum total of all results:", expressionTotal)

Sum total of all results: 18213007238947


## PART 2

In [7]:
## The easiest (well, less laborious) way to implement the inverse PEMDAS order  with the code we've already written
## is to first prune the sums, then process the expression as before
def ProcessAdvancedExpression(expr):
    while True:
        ## When all sums have been purged, the previous algorithm will work just fine
        nextSumIndex = next((index for index, element in enumerate(expr) if element == '+'), -1)
        if nextSumIndex == -1:
            return ProcessLongformExpression(expr)
        
        ## If we encounter a + and it's numbers left and right... Well, it's a sum
        if (expr[nextSumIndex-1].isdigit() and expr[nextSumIndex+1].isdigit()):
            summation = int(expr[nextSumIndex-1]) + int(expr[nextSumIndex+1])
            expr = expr[:nextSumIndex-1] + [str(summation)] + expr[nextSumIndex+2:]
        ## Here we consider the case (expr1) + (expr2)
        ## Note we process the right term first, so as not to fuck up indexing on the left side
        else:
            if expr[nextSumIndex+1] == '(':
                indexOfMatchingRightParenthesis = RightParenthesisIndex(expr, nextSumIndex+1)
                rightSummation = ProcessAdvancedExpression(expr[nextSumIndex+1:indexOfMatchingRightParenthesis+1])
                expr = expr[:nextSumIndex+1] + [str(rightSummation)] + expr[indexOfMatchingRightParenthesis+1:]
            if expr[nextSumIndex-1] == ')':
                indexOfMatchingLeftParenthesis = LeftParenthesisIndex(expr, nextSumIndex-1)
                leftSummation = ProcessAdvancedExpression(expr[indexOfMatchingLeftParenthesis:nextSumIndex])
                expr = expr[:indexOfMatchingLeftParenthesis] + [str(leftSummation)] + expr[nextSumIndex:]

In [8]:
advancedExpressionTotal = 0

for expression in expressionList:
    advancedExpressionTotal += ProcessAdvancedExpression(expression)

print("Sum total of all results with advanced math:", advancedExpressionTotal)

Sum total of all results with advanced math: 388966573054664
